In [ ]:
#Only use if sandbox is not installed
import os,sys
sys.path.append('../../../')

import warnings
warnings.filterwarnings("ignore")    



## Before starting be sure you have some models in the checkpoints folder 

In [ ]:
from sandbox import _test_data
if len(os.listdir(_test_data["landscape_generation"] + "checkpoints")) == 0:
    from warnings import warn 
    warn('No models to run! You need to download the files using Pooch' 
         '(Check README file "Downloading sample datasets")')
    
    from sandbox.utils.download_sample_datasets import (landscape_models, 
                                                        download_landscape_name)
    
    while True:
        if input("Do you want to download an specific Trained model? [y/n]") == "y":
            print("Available models: %s" % landscape_models)
            model = input("Name of model to download:")
            download_landscape_name(model)
        else:
            break
else:
    print("Available models to run: \n{}".format(os.listdir(_test_data["landscape_generation"] + "checkpoints")))

## Initialize Sensor, Projector, marker detection and main thread

In [ ]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"


#Import Sensor
from sandbox.sensor import Sensor
sensor = Sensor(calibsensor=_calibsensor, name="dummy")

# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)

# Initialize marker detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor = sensor)


# Initialize main thread for live update
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco)

# Run main thread
main.run()


In [ ]:
main.run()

In [ ]:
# Path to pytorch
from sandbox import _package_dir
package_dir=os.path.abspath(_package_dir+"/../../pytorch-CycleGAN-and-pix2pix")

# Import the desired module
from sandbox.modules.pytorch import LandscapeGeneration
landscape = LandscapeGeneration(extent=sensor.extent, package_dir=package_dir)

import os
from sandbox import _test_data
os.listdir(_test_data["landscape_generation"] + "checkpoints")

# Add module to main thread
main.add_module('landscape', landscape)

In [ ]:
assert len(landscape.name_trained_models)>0
landscape.name_model = landscape.name_trained_models[0]

#Now you can select which model you want to select, by default it will use the model that ppears first in the list
print("Possible models: ", landscape.name_trained_models, "\n Model to use:", landscape.name_model)

In [ ]:
# Save snapchot of topography for a Pix2Pix model
save_topo = landscape.LoadArea.extractTopo()
assert landscape.LoadArea.absolute_topo is not None

DEM = landscape.get_image_modify()

landscape.save_image(DEM, name = 'landscape_image.png')

In [ ]:
# Save snapchot of topography for a CycleGAN model
save_topo = landscape.LoadArea.extractTopo()
assert landscape.LoadArea.absolute_topo is not None

DEM = landscape.get_image_modify_cycleGAN()

landscape.save_image_cycleGAN(DEM, name = 'landscape_image.png')

## Next command gives you the command to execute in the command line:

In [ ]:
landscape.run_cmd()#package_dir=package_dir)

## you can also run it inside a cell like so:

In [ ]:
# For example: !python A:\\CGRE\\pytorch-CycleGAN-and-pix2pix/test.py --dataroot A:\\CGRE\\open_AR_Sandbox\\notebooks\\tutorials\\09_LandscapeGeneration\\saved_DEMs --results_dir A:\\CGRE\\open_AR_Sandbox\\notebooks\\tutorials\\09_LandscapeGeneration\\results --checkpoints_dir A:\\CGRE\\open_AR_Sandbox\\notebooks\\tutorials\\09_LandscapeGeneration\\checkpoints --name Himalaya10x10_gist --model pix2pix --gpu_ids -1 --direction AtoB


In [ ]:
landscape.read_result(name = 'landscape_image.png')

In [ ]:
import matplotlib.pyplot as plt
main.stop()
fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(landscape.DEM, origin="lower", cmap = "gist_earth")
ax[0].set_title("Real DEM Image")
ax[0].set_axis_off()

ax[1].imshow(landscape.img,  extent=landscape.LoadArea.to_box_extent)
ax[1].set_title("Generated Landscape Image")
ax[1].set_axis_off()
plt.show()
main.run()

In [ ]:
landscape.show_landscape=True